In [26]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import OneHotEncoder

def extract_dataset(main_folder, image_size=(48, 48)):
    train_dataset = []
    train_labels = []
    val_dataset = []
    val_labels = []

    # Define a dictionary to map each emotion to a unique label
    emotion_labels = {'neutral': 0, 'happy': 1, 'angry': 2, 'surprise': 3, 'sad': 4}

    # Loop through emotions in the main folder
    for emotion in emotion_labels:
        train_folder = os.path.join(main_folder, 'train', emotion)
        val_folder = os.path.join(main_folder, 'validation', emotion)
        label = emotion_labels[emotion]

        # Training set
        for filename in os.listdir(train_folder):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
                image_path = os.path.join(train_folder, filename)

                # Load the image using TensorFlow and convert to grayscale
                img = load_img(image_path, color_mode='grayscale', target_size=image_size)
                img_array = img_to_array(img)

                # Normalize the pixel values to the range [0, 1]
                img_array /= 255.0

                # Append the image data and label to the training dataset
                train_dataset.append(img_array)
                train_labels.append(label)

        # Validation set
        for filename in os.listdir(val_folder):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
                image_path = os.path.join(val_folder, filename)

                # Load the image using TensorFlow and convert to grayscale
                img = load_img(image_path, color_mode='grayscale', target_size=image_size)
                img_array = img_to_array(img)

                # Normalize the pixel values to the range [0, 1]
                img_array /= 255.0

                # Append the image data to the validation dataset
                val_dataset.append(img_array)
                val_labels.append(label)

    # Convert datasets to numpy arrays
    train_dataset = np.array(train_dataset)
    train_labels = np.array(train_labels)
    val_dataset = np.array(val_dataset)
    val_labels = np.array(val_labels)

    # One-hot encode the labels
    one_hot_encoder = OneHotEncoder(sparse=False)
    train_labels = one_hot_encoder.fit_transform(train_labels.reshape(-1, 1))
    val_labels = one_hot_encoder.transform(val_labels.reshape(-1, 1))

    return train_dataset, train_labels, val_dataset, val_labels



In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

def create_vggnet16_model(input_shape=(48, 48, 1)):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(2048, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(5, activation='softmax'))

    return model

def train_model_with_simulated_annealing(model, train_dataset, train_labels, val_dataset, val_labels, initial_temp=1000, cooling_rate=0.003, max_iter=1000):
    # 编译模型
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    # 训练模型
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(train_dataset, train_labels, epochs=20, batch_size=64, validation_data=(val_dataset, val_labels), callbacks=[early_stopping])

    # 模拟退火算法优化模型参数
    current_temp = initial_temp
    current_weights = model.get_weights()
    best_weights = current_weights
    best_loss = float('inf')

    for i in range(max_iter):
        # 随机扰动权重
        new_weights = [w + np.random.normal(0, 1, w.shape) for w in current_weights]
        model.set_weights(new_weights)

        # 计算新的损失
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(train_dataset, train_labels, epochs=1, batch_size=64, verbose=0, validation_data=(val_dataset, val_labels))
        new_loss = history.history['val_loss'][0]

        # 判断是否接受新权重
        if new_loss < best_loss:
            best_loss = new_loss
            best_weights = new_weights
        else:
            acceptance_prob = np.exp((best_loss - new_loss) / current_temp)
            if np.random.rand() < acceptance_prob:
                best_loss = new_loss
                best_weights = new_weights

        # 降温
        current_temp *= (1 - cooling_rate)

    model.set_weights(best_weights)

    return model, history


In [31]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

def evaluate_model(model, val_dataset, val_labels, emotion_labels):
    # 模型评估
    val_loss, val_accuracy = model.evaluate(val_dataset, val_labels)
    print(f'Validation loss: {val_loss}')
    print(f'Validation accuracy: {val_accuracy}')

    # 预测验证集
    val_predictions = model.predict(val_dataset)
    val_pred_classes = np.argmax(val_predictions, axis=1)
    val_true_classes = np.argmax(val_labels, axis=1)

    # 分类报告
    report = classification_report(val_true_classes, val_pred_classes, target_names=emotion_labels.keys())
    print(report)

    # 混淆矩阵
    conf_matrix = confusion_matrix(val_true_classes, val_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=emotion_labels.keys(), yticklabels=emotion_labels.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # ROC曲线和AUC
    plt.figure(figsize=(10, 8))
    for i, emotion in enumerate(emotion_labels.keys()):
        fpr, tpr, _ = roc_curve(val_labels[:, i], val_predictions[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{emotion} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

    # 损失和准确率图像
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Over Time')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Time')
    plt.legend()

    plt.show()


In [32]:


# 设置数据集文件夹路径和图像尺寸大小
main_folder_path = 'dataset'
image_size = (48, 48)

# 提取数据集
train_dataset, train_labels, val_dataset, val_labels = extract_dataset(main_folder_path, image_size=image_size)

# 使用预定义模型
model = create_vggnet16_model()
# 使用模拟退火算法训练模型
trained_model, history = train_model_with_simulated_annealing(model, train_dataset, train_labels, val_dataset, val_labels)

# 评估训练好的模型
emotion_labels = {'neutral': 0, 'happy': 1, 'angry': 2, 'surprise': 3, 'sad': 4}
evaluate_model(trained_model, val_dataset, val_labels, emotion_labels)

# 保存训练好的模型
trained_model.save('trained_model_with_simulated_annealing.h5')


F:\ProgramData\anaconda3\envs\pytorch\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
F:\ProgramData\anaconda3\envs\pytorch\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 582s 2s/step - accuracy: 0.3696 - loss: 6.3679 - val_accuracy: 0.2164 - val_loss: 5.9336
Epoch 2/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 580s 2s/step - accuracy: 0.5432 - loss: 5.0645 - val_accuracy: 0.5752 - val_loss: 4.6921
Epoch 3/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 571s 2s/step - accuracy: 0.6139 - loss: 4.5234 - val_accuracy: 0.5968 - val_loss: 4.2810
Epoch 4/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.6721 - loss: 4.0296 - val_accuracy: 0.6098 - val_loss: 3.9006
Epoch 5/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 569s 2s/step - accuracy: 0.7201 - loss: 3.5704 - val_accuracy: 0.6274 - val_loss: 3.5724
Epoch 6/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 568s 2s/step - accuracy: 0.7625 - loss: 3.1680 - val_accuracy: 0.6474 - val_loss: 3.2802
Epoch 7/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.8132 - loss: 2.7924 - val_accuracy: 0.6583 - val_loss: 3.0383
Epoch 8/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 564s 1s/step - accuracy: 0.8581 - loss: 2.4584 - val_accu

KeyboardInterrupt: 